In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#This notebook is created to run value analysis with your own model on multi-class datasets
#specify the required info here and run the notebook to receive value analysis result for your model
modelName = 'name_of_your_model'
resPath = 'define_the_path_for_results'
data_folder = 'define_the_path_where_you_keep_the_confidence_values_of_your_model_and_the_datasets'
confidencesToVal = 'name_of_the_numpy_array_for_the_confidences_on_validation_set.npy'
dataToVal = 'name_of_validation_set.csv'
confidencesToTest = 'name_of_the_numpy_array_for_the_confidences_on_test_set.npy'
dataToTest ='name_of_test_set.csv'
ground_truth_column = 'specify_the_column_for_ground_truth_in_your_csv_files'
txt = 'specify_the_column_for_text_in_your_csv_files'
datasetName = 'name_of_your_dataset'

In [ ]:
#result file
logfile_name = "{}_{}".format(datasetName)

#cost-based parameters
Vr = 0.0
Vc = 1.0
Vw_g = list(np.arange(0, -10.1, -1))
t_g = list(np.arange(0, 1.01, 0.01))

In [ ]:
logits_val = np.load(data_folder  + confidencesToVal)
y_val_df = pd.read_csv(data_folder + dataToVal)
y_val = y_val_df[ground_truth_column].values

logits_test = np.load(data_folder  + confidencesToTest)
y_test_df = pd.read_csv(data_folder + dataToTest)
y_test = y_test_df[ground_truth_column].values

In [ ]:
def cost_based_threshold(k):
    t = (k)/(k+1)
    return t

def calculate_value(y_hat_proba, y, t, Vr, Vc, Vw):

    y_pred = np.array([np.where(l == np.amax(l))[0][0] if (np.amax(l) > t) else -1 for l in y_hat_proba])

    # now lets compute the actual value of each prediction
    
    value_vector = np.full(y_pred.shape[0], Vc)

    value_vector[(y_pred != y) & (y_pred != -1)] = Vw
    
    #loss due to asking humans
    value_vector[y_pred == -1] = Vr
    value = np.sum(value_vector) / len(y)

    numOfRejectedSamples = np.count_nonzero(y_pred == -1)
    numOfWrongPredictions = np.count_nonzero((y_pred != y) & (y_pred != -1))
    return value, numOfRejectedSamples, numOfWrongPredictions

def find_optimum_confidence_threshold(y_hat_proba, y, t_list, Vr, Vc, Vw):

    cost_list = {}

    for t in t_list:
        value, _ , __ = calculate_value(y_hat_proba, y, t, Vr, Vc, Vw)
        cost_list["{}".format(t)] = value
    # find t values with maximum value
    maxValue = max(cost_list.values())
    optTList = [float(k) for k, v in cost_list.items() if v == maxValue]
    # pick the one with the lowest confidence
    optimumT = min(optTList)

    return optimumT, cost_list 

#cost based calibration analysis
def cost_based_analysis(y_hat_proba_val, y_val, y_hat_proba_test, y_test, res_path, logfile_name, Vr, Vc, Vw_list, confT_list):

    # create log file
    rc_path = res_path + logfile_name + "_costBased_test.csv"
    with open(rc_path, 'w') as f:
        c = 'Vr, Vc, Vw, k, t, value, rejected, wrong, t_optimal, value_optimal, rejected_opt, wrong_opt'
        f.write(c + '\n')

    for Vw in Vw_list:
        k = (-1)*(Vw / Vc)
        t = cost_based_threshold(k)
        value_test, rej_test, wrong_test = calculate_value(y_hat_proba_test, y_test, t, Vr, Vc, Vw)

        t_optimal, cost_list = find_optimum_confidence_threshold(y_hat_proba_val, y_val, confT_list, Vr, Vc, Vw)
        value_test_opt, rej_test_opt, wrong_test_opt = calculate_value(y_hat_proba_test, y_test, t_optimal, Vr, Vc, Vw)

        with open(rc_path, 'a') as f:
            res_i = '{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}\n'.format(Vr, Vc, Vw, k, t, value_test, rej_test, wrong_test, t_optimal, value_test_opt, rej_test_opt, wrong_test_opt)
            f.write(res_i)  

In [ ]:
cost_based_analysis(logits_val, y_val, logits_test, y_test, res_path, logfile_name, Vr, Vc, Vw_g, t_g)